In [ ]:
import carla
import random
import math
import time
import os
import argparse
import sys
import keyboard

def run(
        map='Town10HD_Opt',  # Name of the map
        radius=10,  # Radius of the spectator rotation circle
        height=5,  # Height of the spectator
        spawn_point=0,  # Index of the spawn point: 0-154 (Town10HD_Opt)
        rotation_speed_degree = 10.0,  # Define the speed of the rotation (degree)
        vehicle_blueprint_id='vehicle.audi.etron_adv',  # Blueprint ID of the vehicle
        sensor_blueprint_id='sensor.camera.rgb',  # Blueprint ID of the sensor 
        image_width='800',  # Define the width of the image
        image_height='600',  # Define the height of the image
        output_dir = r'D:\Files\Code\Data\Benchmarking_Physical_Attack',  # Define the output directory
        save_images = True,  # Define if the images should be saved
        speed_weather_changing = 10.0,  # Define the speed of the weather changing
        total_rotation_degree = 360  # Define the total rotation degree
):
    # Connect to the client and retrieve the world object
    client = carla.Client('localhost', 2000)
    world = client.load_world(map)

    # Get the blueprint library and filter for the vehicle blueprints
    vehicle_blueprint = world.get_blueprint_library().find(vehicle_blueprint_id)

    # Choose a spawn location
    # In this example, we're spawningradius of the circle the vehicle at a random location
    spawn_points = world.get_map().get_spawn_points()  # len(transforms) = 155 for Town10HD_Opt
    vehicle_transform = spawn_points[spawn_point]

    # Spawn the vehicle
    vehicle = world.spawn_actor(vehicle_blueprint, vehicle_transform)

    angle_radian = 0.0  # Initial angle_radian of the spectator
    angle_degree = 0.0  # Initial angle_degree of the spectator

    # Get the spectator
    spectator = world.get_spectator()

    # Create a blueprint for the camera
    camera_blueprint = world.get_blueprint_library().find(sensor_blueprint_id)

    # Set the resolution of the camera
    camera_blueprint.set_attribute('image_size_x', image_width)
    camera_blueprint.set_attribute('image_size_y', image_height)

    # Attach the camera to the spectator
    camera_initial_transform = carla.Transform(carla.Location(x=0.0, y=0.0, z=0.0))  # The camera's transform relative to the spectator

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # weather_update_freq = 0.1 / speed_weather_changing
    weather = Weather(world.get_weather())
    # elapsed_time = 0.0

    # Name the output directory with the rotation speed and the weather speed
    output_dir = os.path.join(output_dir, 'output' + '_rs%02.2f' % rotation_speed_degree + '_ws%02.2f' % speed_weather_changing)
    # print(output_dir)

    time.sleep(2)  # Wait for the car landing before taking the first image

    while angle_degree < total_rotation_degree:
        # Update the weather
        weather.tick(speed_weather_changing)
        world.set_weather(weather.weather)
        sys.stdout.write('\r' + str(weather) + 12 * ' ')
        sys.stdout.flush()

        # Calculate the transform of the spectator
        transform = rotation_transform_update(vehicle, radius, height, angle_radian)
        # Set the transform of the spectator
        spectator.set_transform(transform)

        weather_string = str(weather).replace(' ', '_').replace(':', '').replace(',', '').replace('(', '_').replace(')', '').replace('=', '').replace('%', '')
        output_path = os.path.join(output_dir, map + '_R%02d' % radius + '_H%02d_' % height + weather_string + '_%06d.png' % angle_degree)

        if save_images:
            camera = world.spawn_actor(camera_blueprint, camera_initial_transform, attach_to=spectator)
            camera.listen(lambda image: save_image(image, output_path, angle_degree))
            generated = os.path.isfile(output_path)
            while not generated:
                generated = os.path.isfile(output_path)
            camera.destroy()
        else:
            # Wait for a while to show the image if no need to save the image
            time.sleep(0.01)
            # Check if 'q' is pressed
            if keyboard.is_pressed('q'):  
                print('You pressed q, loop will break')
                break  # exit loop

        # Calculate the angle_radian based on the speed
        angle_radian += (math.pi * 2.0 / 360.0) * rotation_speed_degree
        angle_degree += rotation_speed_degree
        
    vehicle.destroy()

# Define a function that calculates the transform of the spectator
def rotation_transform_update(vehicle, radius, height, angle_radian):
    # Get the location of the vehicle
    vehicle_location = vehicle.get_location()

    # Calculate the new location of the spectator
    location = carla.Location()
    location.x = vehicle_location.x - radius * math.cos(angle_radian)  # X-coordinate
    location.y = vehicle_location.y + radius * math.sin(angle_radian)  # Y-coordinate
    location.z = vehicle_location.z + height                    # Z-coordinate

    # Calculate the rotation that makes the spectator look at the vehicle
    rotation = carla.Rotation()
    rotation.yaw = -math.degrees(angle_radian)  # Yaw angle_radian
    rotation.pitch = -math.degrees(math.atan(height / radius))  # Pitch angle_radian

    # Create a new transform with the new location and the new rotation
    transform = carla.Transform(location, rotation)
    return transform

# Define a function to save images
def save_image(image, output_path, angle_degree):
    # Save the image to disk
    image.save_to_disk(output_path)

def clamp(value, minimum=0.0, maximum=100.0):
    return max(minimum, min(value, maximum))

class Sun(object):
    def __init__(self, azimuth, altitude):
        self.azimuth = azimuth
        self.altitude = altitude
        self._t = 0.0

    def tick(self, delta_seconds):
        self._t += 0.008 * delta_seconds
        self._t %= 2.0 * math.pi
        self.azimuth += 0.25 * delta_seconds
        self.azimuth %= 360.0
        self.altitude = (70 * math.sin(self._t)) - 20

    def __str__(self):
        return 'Sun(alt: %.2f, azm: %.2f)' % (self.altitude, self.azimuth)
    
class Storm(object):
    def __init__(self, precipitation):
        self._t = precipitation if precipitation > 0.0 else -50.0
        self._increasing = True
        self.clouds = 0.0
        self.rain = 0.0
        self.wetness = 0.0
        self.puddles = 0.0
        self.wind = 0.0
        self.fog = 0.0

    def tick(self, delta_seconds):
        delta = (1.3 if self._increasing else -1.3) * delta_seconds
        self._t = clamp(delta + self._t, -250.0, 100.0)
        self.clouds = clamp(self._t + 40.0, 0.0, 90.0)
        self.rain = clamp(self._t, 0.0, 80.0)
        delay = -10.0 if self._increasing else 90.0
        self.puddles = clamp(self._t + delay, 0.0, 85.0)
        self.wetness = clamp(self._t * 5, 0.0, 100.0)
        self.wind = 5.0 if self.clouds <= 20 else 90 if self.clouds >= 70 else 40
        self.fog = clamp(self._t - 10, 0.0, 30.0)
        if self._t == -250.0:
            self._increasing = True
        if self._t == 100.0:
            self._increasing = False

    def __str__(self):
        return 'Storm(clouds=%d%%, rain=%d%%, wind=%d%%)' % (self.clouds, self.rain, self.wind)
    
class Weather(object):
    def __init__(self, weather):
        self.weather = weather
        self._sun = Sun(weather.sun_azimuth_angle, weather.sun_altitude_angle)
        self._storm = Storm(weather.precipitation)

    def tick(self, delta_seconds):
        self._sun.tick(delta_seconds)
        self._storm.tick(delta_seconds)
        self.weather.cloudiness = self._storm.clouds
        self.weather.precipitation = self._storm.rain
        self.weather.precipitation_deposits = self._storm.puddles
        self.weather.wind_intensity = self._storm.wind
        self.weather.fog_density = self._storm.fog
        self.weather.wetness = self._storm.wetness
        self.weather.sun_azimuth_angle = self._sun.azimuth
        self.weather.sun_altitude_angle = self._sun.altitude

    def __str__(self):
        return '%s %s' % (self._sun, self._storm)

def main():
    run()

if __name__ == '__main__':
    main()
